In [1]:
#from google.colab import files
#uploaded = files.upload()

In [2]:
#file_names =[]
#for file in uploaded.keys():
#  file_names.append(file)

In [3]:
import os
path = "../data/"
file_list = os.listdir( path )

remove_files = ['input.txt','LICENSE']
for r_files in remove_files:
    file_list.remove(r_files)

file_list = file_list[:13]
file_list

['408.txt',
 '104.txt',
 '301.txt',
 '405.txt',
 '407.txt',
 '310.txt',
 '211.txt',
 '411.txt',
 '303.txt',
 '203.txt',
 '304.txt',
 '107.txt',
 '309.txt']

In [4]:
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
    process = psutil.Process(os.getpid())
    print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
    print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

  Running setup.py bdist_wheel for gputil ... done
  Stored in directory: /content/.cache/pip/wheels/17/0f/04/b79c006972335e35472c0b835ed52bfc0815258d409f560108
Successfully built gputil
  Running setup.py bdist_wheel for humanize ... done
  Stored in directory: /content/.cache/pip/wheels/69/86/6c/f8b8593bc273ec4b0c653d3827f7482bb2001a2781a73b7f44
Successfully built humanize
Gen RAM Free: 12.4 GB  | Proc size: 142.1 MB
GPU RAM Free: 11439MB | Used: 0MB | Util   0% | Total 11439MB


## Import Dependency

In [5]:
#import Keras library
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM, Input, Bidirectional
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.metrics import categorical_accuracy

#import spacy, and spacyfrench model
# spacy is used to work on text
! pip install spacy
import spacy
!python -m spacy download fr
nlp = spacy.load('fr')

#import other libraries
import numpy as np
import random
import sys
import os
import time
import codecs
import collections
from six.moves import cPickle

Using TensorFlow backend.


    100% |████████████████████████████████| 39.8MB 1.0MB/s 
  Running setup.py install for fr-core-news-sm ... done

    Linking successful
    /usr/local/lib/python3.6/dist-packages/fr_core_news_sm -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/fr

    You can now load the model via spacy.load('fr')



In [6]:
#define parameters used in the tutorial
data_dir = '../data'# data directory containing input.txt
save_dir = '../out' # directory to store models
seq_length = 30 # sequence length
sequences_step = 1 #step to create sequences
vocab_file = os.path.join(save_dir,  "words_vocab.pkl")

In [7]:
def create_wordlist(doc):
    wl = []
    for word in doc:
        if word.text not in ("\n","\n\n",'\u2009','\xa0'):
            wl.append(word.text.lower())
    return wl

In [8]:
wordlist = []

for file_name in file_list:
    input_file = os.path.join(data_dir, file_name)
    #read data
    with codecs.open(input_file, "r") as f:
        data = f.read()
    #create sentences
    doc = nlp(data)
    wl = create_wordlist(doc)
    wordlist = wordlist + wl

In [9]:
print(wordlist[:20])
printm()

['mara', 'est', 'revenue', 'dans', 'la', 'caverne', 'et', 's’', 'est', 'faufilée', 'dans', 'la', 'partie', 'transformée', 'en', 'hôpital', 'de', 'campagne', '.', 'elle']
Gen RAM Free: 11.6 GB  | Proc size: 1.6 GB
GPU RAM Free: 11439MB | Used: 0MB | Util   0% | Total 11439MB


In [10]:
len(wordlist)

47213

In [11]:
vocab_file = os.path.join(save_dir, "words_vocab.pkl")

In [12]:
# count the number of words
word_counts = collections.Counter(wordlist)

# Mapping from index to word : that's the vocabulary
vocabulary_inv = [x[0] for x in word_counts.most_common()]
vocabulary_inv = list(sorted(vocabulary_inv))

# Mapping from word to index
vocab = {x: i for i, x in enumerate(vocabulary_inv)}
words = [x[0] for x in word_counts.most_common()]

#size of the vocabulary
vocab_size = len(words)
print("vocab size: ", vocab_size)

#save the words and vocabulary
with open(os.path.join(vocab_file), 'wb') as f:
    cPickle.dump((words, vocab, vocabulary_inv), f)

vocab size:  5749


In [13]:
printm()

Gen RAM Free: 11.6 GB  | Proc size: 1.6 GB
GPU RAM Free: 11439MB | Used: 0MB | Util   0% | Total 11439MB


In [14]:
#create sequences
sequences = []
next_words = []
for i in range(0, len(wordlist) - seq_length, sequences_step):
    sequences.append(wordlist[i: i + seq_length])
    next_words.append(wordlist[i + seq_length])

print('nb sequences:', len(sequences))

nb sequences: 47183


In [15]:
X = np.zeros((len(sequences), seq_length, vocab_size), dtype=np.bool)
y = np.zeros((len(sequences), vocab_size), dtype=np.bool)
for i, sentence in enumerate(sequences):
    for t, word in enumerate(sentence):
        X[i, t, vocab[word]] = 1
    y[i, vocab[next_words[i]]] = 1

In [16]:
printm()

Gen RAM Free: 3.5 GB  | Proc size: 9.8 GB
GPU RAM Free: 11439MB | Used: 0MB | Util   0% | Total 11439MB


In [17]:
def bidirectional_lstm_model(seq_length, vocab_size):
    print('Build LSTM model.')
    model = Sequential()
    model.add(Bidirectional(LSTM(rnn_size, activation="relu"),input_shape=(seq_length, vocab_size)))
    model.add(Dropout(0.6))
    model.add(Dense(vocab_size))
    model.add(Activation('softmax'))
    
    optimizer = Adam(lr=learning_rate)
    callbacks=[EarlyStopping(patience=2, monitor='val_loss')]
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=[categorical_accuracy])
    return model

In [18]:
rnn_size = 256 # size of RNN
batch_size = 32 # minibatch size
seq_length = 30 # sequence length
num_epochs = 2 # number of epochs
learning_rate = 0.001 #learning rate
sequences_step = 1 #step to create sequences

In [19]:
md = bidirectional_lstm_model(seq_length, vocab_size)
md.summary()

Build LSTM model.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, 512)               12300288  
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 5749)              2949237   
_________________________________________________________________
activation_1 (Activation)    (None, 5749)              0         
Total params: 15,249,525
Trainable params: 15,249,525
Non-trainable params: 0
_________________________________________________________________


In [20]:
#fit the model
callbacks=[EarlyStopping(patience=4, monitor='val_loss'),
           ModelCheckpoint(filepath=save_dir + "/" + 'my_model_gen_sentences_lstm.{epoch:02d}-{val_loss:.2f}.hdf5',\
                           monitor='val_loss', verbose=0, mode='auto', period=2)]
history = md.fit(X, y,
                 batch_size=batch_size,
                 shuffle=True,
                 epochs=num_epochs,
                 callbacks=callbacks,
                 validation_split=0.01)

Train on 46711 samples, validate on 472 samples
Epoch 1/2
46711/46711 [==============================] - 431s 9ms/step - loss: 6.3761 - categorical_accuracy: 0.0475 - val_loss: 5.9715 - val_categorical_accuracy: 0.0530
Epoch 2/2
46711/46711 [==============================] - 429s 9ms/step - loss: 5.9608 - categorical_accuracy: 0.0618 - val_loss: 5.7778 - val_categorical_accuracy: 0.0805


In [21]:
printm()

Gen RAM Free: 3.3 GB  | Proc size: 10.1 GB
GPU RAM Free: 11439MB | Used: 0MB | Util   0% | Total 11439MB


In [31]:
#save the model
md.save(save_dir + "/" + 'my_model_gen_sentences_lstm.final.hdf5')

In [32]:
#load vocabulary
print("loading vocabulary...")
vocab_file = os.path.join(save_dir, "words_vocab.pkl")

with open(os.path.join(save_dir, 'words_vocab.pkl'), 'rb') as f:
        words, vocab, vocabulary_inv = cPickle.load(f)

vocab_size = len(words)

loading vocabulary...


In [33]:
printm()

Gen RAM Free: 3.1 GB  | Proc size: 10.4 GB
GPU RAM Free: 11439MB | Used: 0MB | Util   0% | Total 11439MB


In [34]:
from keras.models import load_model
# load the model
print("loading model...")
model = load_model(save_dir + "/" + 'my_model_gen_sentences_lstm.final.hdf5')

loading model...


In [35]:
printm()

Gen RAM Free: 2.6 GB  | Proc size: 10.7 GB
GPU RAM Free: 11439MB | Used: 0MB | Util   0% | Total 11439MB


In [36]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [37]:
#initiate sentences
seed_sentences = "nolan avance sur le chemin de pierre et grimpe les marches ."
generated = ''
sentence = []
for i in range (seq_length):
    sentence.append("a")

seed = seed_sentences.split()

for i in range(len(seed)):
    sentence[seq_length-i-1]=seed[len(seed)-i-1]

generated += ' '.join(sentence)
print('Generating text with the following seed: "' + ' '.join(sentence) + '"')

print()

Generating text with the following seed: "a a a a a a a a a a a a a a a a a a nolan avance sur le chemin de pierre et grimpe les marches ."

